# PHASE 3: Molecule Properties
"""

PHASE 3: Determine physicochemical properties of the new molecules 

Created on Thursday May 26 2023 
Updated on Wednesday June 07 2023
Updated on Monday July 17 2023 - used new list of 10k new mols 

@author: Odifentse M Lehasa

The purpose of this notebook is to determine the physicochemical properties of the new molecules, so as to see if these are drug-like (good oral drug candidates).

In the previous phase, a set of 10,000 new molecules were generated.

In this phase, those new molecules will be screened for their physicochemical properties. 

"""

## STEP 0: IMPORT LIBRARIES

In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import BRICS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Lipinski
from rdkit.Chem import QED as QED
from rdkit.Chem import Recap as Recap
import pandas as pd

## STEP 1: CALCULATE THE PHYSICOCHEMICAL PROPERTIES OF THE NEW MOLECULES

### View dataframe of new molecules

In [6]:
# Use dataset of new molecules from phase 2.

# View dataset of new molecules from phase 2.

df_newMols = pd.read_csv('/Users/odilehasa/Hypertension/Final_Experiments/FINAL - October/Output/2. C09_new mols (10,000).csv', index_col=0)

df_newMols

,0
0,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...
1,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...
2,CO[C@@H]1C[C@H]2CCCC[C@@H]2N1[C@@H](CCCCN)C(=O...
3,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...
4,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...
...,...
9995,CCCCC1=NC2(CCCC2)C(=O)N1[C@@H]1CC2(CN1[C@@H](C...
9996,CO[C@@H]1CC2(CN1[C@@H](CCCCN)C(=O)N1[C@H](c3no...
9997,NCCCC[C@@H](C(=O)N1[C@H](c2noc(=O)[nH]2)C[C@@H...
9998,NCCCC[C@@H](C(=O)N1[C@H](c2noc(=O)[nH]2)C[C@@H...


### Calculate the physicochemical properties 

In [10]:
# create a list where all the results will be stored
properties_list= list()

# calculate the properties

for z in range(len(df_newMols)):
    x = Chem.MolFromSmiles(df_newMols['0'][z])

    # Determine number of aromatic and aliphatic rings
    aromatic = Lipinski.NumAromaticRings(x)
    aliphatic = Lipinski.NumAliphaticRings(x)
    
    # Lipinski rule of 5 properties
    Mol_weight = Descriptors.MolWt(x)
    LogP = Descriptors.MolLogP(x)                       # lipophilicity 
    Hdonors = rdMolDescriptors.CalcNumLipinskiHBD(x)    # Hydrogen bond donors
    Hacceptors = rdMolDescriptors.CalcNumLipinskiHBA(x) # Hydrogen bond acceptors
    
    # Ro5 Test  
    if (Mol_weight <= 500) & (LogP <= 5) & (Hdonors <= 5) & (Hacceptors <=10): 
        Ro5_druggable = 1   #1 for True (meet criteria)
    else:
        Ro5_druggable = 0   # 0 for False (does not meet criteria)
    
    # Other Physicochemical properties
    Exact_mol_weight = Descriptors.ExactMolWt(x)
    Rotate_bonds = Lipinski.NumRotatableBonds(x)
    heavy_atoms = Descriptors.HeavyAtomCount(x)
    qed = QED.weights_mean(x)                
    prop_forcast_index = LogP+aromatic
    PSA = QED.properties(x)[4]
    
    # Physicochemical Test  
    if (heavy_atoms <38) & (PSA <=140) & (Rotate_bonds <=10) & (aromatic < 4) & (qed <=1) & (prop_forcast_index < 7):
        physico_druggable = 1
    else:
        physico_druggable = 0
    
    
    
  # combine above results
    properties_total = (Chem.MolToSmiles(x),aromatic,aliphatic,Mol_weight,Exact_mol_weight,LogP,Hdonors,Hacceptors,Rotate_bonds,
                       heavy_atoms,qed,prop_forcast_index,PSA, Ro5_druggable, physico_druggable)
    properties_list.append(properties_total) 


# save list as dataframe
df_props = pd.DataFrame(properties_list, columns =['Canonical SMILES', 'Aromatic Rings (No.)', 'Aliphatic Rings (No.)',
                                                             'AVG Molecular weight','Exact Molecular weight','LogP','Hdonors',
                                                             'Hacceptors','Rotatable bonds','Heavy Atoms (No.)','QED',
                                                             'Property Forecast Index','PSA','Druggable (Lipinski)', 'Druggable (Physicochemical)'])

df_props.to_csv('3. New molecules and physicochemical properties.csv')
df_props


,Canonical SMILES,Aromatic Rings (No.),Aliphatic Rings (No.),AVG Molecular weight,Exact Molecular weight,LogP,Hdonors,Hacceptors,Rotatable bonds,Heavy Atoms (No.),QED,Property Forecast Index,PSA,Druggable (Lipinski),Druggable (Physicochemical)
0,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...,0,5,473.746,473.398128,4.8494,3,5,8,34,0.494569,4.8494,69.80,1,1
1,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...,1,5,608.824,608.393771,3.9493,4,9,10,44,0.343457,4.9493,127.41,0,0
2,CO[C@@H]1C[C@H]2CCCC[C@@H]2N1[C@@H](CCCCN)C(=O...,0,4,421.626,421.330442,2.8730,3,6,8,30,0.589344,2.8730,79.03,1,1
3,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...,1,4,475.634,475.315855,2.3181,4,9,8,34,0.492189,3.3181,128.69,1,1
4,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...,1,4,473.727,473.307599,4.7033,3,5,8,33,0.536939,5.7033,69.80,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,CCCCC1=NC2(CCCC2)C(=O)N1[C@@H]1CC2(CN1[C@@H](C...,1,6,659.923,659.328745,4.2544,3,11,11,45,0.333649,5.2544,141.13,0,0
9996,CO[C@@H]1CC2(CN1[C@@H](CCCCN)C(=O)N1[C@H](c3no...,1,4,497.687,497.213047,2.1571,3,9,8,33,0.520950,3.1571,117.69,1,1
9997,NCCCC[C@@H](C(=O)N1[C@H](c2noc(=O)[nH]2)C[C@@H...,2,4,551.695,551.198459,1.6022,4,12,8,37,0.407653,3.6022,167.35,0,0
9998,NCCCC[C@@H](C(=O)N1[C@H](c2noc(=O)[nH]2)C[C@@H...,2,4,549.788,549.190203,3.9874,3,8,8,36,0.476140,5.9874,108.46,0,1


## STEP 2: FILTER OUT MOLECULES THAT FAILED CRITERIA

In [11]:

# Remove those molecules that did not pass the tests - Ro5 and Physicochemical criteria

# Ro5
Ro5druggable = df_props[df_props['Druggable (Lipinski)'] == 1] # 1 is pass (meets criteria) and 0 is fail (does not meet criteria)
Ro5druggable_df = Ro5druggable

# Physicochemical
Physicodruggable = Ro5druggable_df[Ro5druggable_df['Druggable (Physicochemical)'] == 1] # 1 is pass (meets criteria) and 0 is fail (does not meet criteria)
Physicodruggable_df = Physicodruggable

# Save the new dataframe of new molecules that meet criteria as a CSV file
Physicodruggable_df.to_csv('3.1. New molecules - passed property criteria.csv')
Physicodruggable_df


,Canonical SMILES,Aromatic Rings (No.),Aliphatic Rings (No.),AVG Molecular weight,Exact Molecular weight,LogP,Hdonors,Hacceptors,Rotatable bonds,Heavy Atoms (No.),QED,Property Forecast Index,PSA,Druggable (Lipinski),Druggable (Physicochemical)
0,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...,0,5,473.746,473.398128,4.8494,3,5,8,34,0.494569,4.8494,69.80,1,1
2,CO[C@@H]1C[C@H]2CCCC[C@@H]2N1[C@@H](CCCCN)C(=O...,0,4,421.626,421.330442,2.8730,3,6,8,30,0.589344,2.8730,79.03,1,1
3,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...,1,4,475.634,475.315855,2.3181,4,9,8,34,0.492189,3.3181,128.69,1,1
4,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...,1,4,473.727,473.307599,4.7033,3,5,8,33,0.536939,5.7033,69.80,1,1
5,NCCCC[C@@H](C(=O)N1[C@H](CO)C[C@H]2CCCC[C@@H]2...,1,4,459.639,459.332174,2.1549,4,9,8,33,0.508852,3.1549,124.26,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9976,CC(C)(O)[C@@H]1C[C@@H]2CCCC[C@@H]2N1[C@@H](CCC...,1,4,489.661,489.331505,2.7066,4,9,8,35,0.478929,3.7066,128.69,1,1
9977,NCCCC[C@@H](C(=O)N1[C@H](c2noc(=O)[nH]2)C[C@@H...,1,4,461.607,461.300205,1.9280,4,9,8,33,0.504578,2.9280,128.69,1,1
9979,CO[C@@H]1C[C@@H]2CCCC[C@@H]2N1[C@@H](CCCCN)C(=...,1,4,461.607,461.300205,2.5396,3,9,8,33,0.571290,3.5396,117.69,1,1
9994,NCCCC[C@@H](C(=O)N1[C@H](c2noc(=O)[nH]2)C[C@@H...,1,4,497.687,497.213047,1.5455,4,9,8,33,0.458140,2.5455,128.69,1,1


# --- END HERE --- 
